In [16]:
#setting up
#import boto3, re
#from sagemaker import get_execution_role
#role = get_execution_role()
from sagemaker import get_execution_role

# Specify the IAM role ARN explicitly
role_arn = 'arn:aws:iam::211125471422:role/maker'

# Use the specified IAM role ARN
role = role_arn




In [17]:
import sagemaker

# Set SageMaker SDK configurations programmatically
sagemaker.Session().config = {"region_name": "eu-north-1", "profile_name": "Sheethal"}

# Initialize SageMaker SDK with custom configurations
sagemaker_session = sagemaker.Session()


In [1]:
#speed up execution of tensorflow operations
import keras
from keras.models import model_from_json
import tensorflow as tf

tf.compat.v1.disable_eager_execution()


In [2]:
!mkdir keras_model

A subdirectory or file keras_model already exists.


In [3]:
ls keras_model

 Volume in drive C is OS
 Volume Serial Number is D409-CDF8

 Directory of c:\Users\mshee\OneDrive\Desktop\proo\keras_model

24-02-2024  09:11    <DIR>          .
03-04-2024  19:50    <DIR>          ..
21-02-2024  22:32        12,137,104 my_model_weights.h5
21-02-2024  22:32             4,370 mymodeljson.json
               2 File(s)     12,141,474 bytes
               2 Dir(s)  230,173,659,136 bytes free


In [2]:
# #loading the model
# json_file = open('C:/Users/mshee/OneDrive/Desktop/proo/keras_model/'+'mymodeljson.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json,custom_objects={"GlorotUniform": tf.keras.initializers.glorot_uniform})


from tensorflow.keras.models import model_from_json

# Load the JSON file
with open('C:/Users/mshee/OneDrive/Desktop/proo/keras_model/'+'mymodeljson.json', 'r') as json_file:
    loaded_model_json = json_file.read()

# Load the model architecture from JSON
loaded_model = model_from_json(loaded_model_json)

# Load the weights (assuming they are saved separately)
loaded_model.load_weights('C:\\Users\mshee\OneDrive\Desktop\proo\keras_model\my_model_weights.h5')
print("Loaded model from disk")
tf.saved_model.save(loaded_model, 'C:\\Users\mshee\OneDrive\Desktop\proo')


Instructions for updating:
Colocations handled automatically by placer.
Loaded model from disk
INFO:tensorflow:Assets written to: C:\Users\mshee\OneDrive\Desktop\proo\assets


INFO:tensorflow:Assets written to: C:\Users\mshee\OneDrive\Desktop\proo\assets


In [3]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants


model_version = '1'
export_dir = 'export/Servo/' + model_version
export_dir

'export/Servo/1'

In [4]:
import shutil
shutil.rmtree(export_dir)

In [5]:
# Build the Protocol Buffer SavedModel at 'export_dir'
build = builder.SavedModelBuilder(export_dir)

In [6]:
import tensorflow as tf

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Assuming loaded_model is your TensorFlow model
# Define the input and output tensors
input_tensor = loaded_model.input
output_tensor = loaded_model.output

# Create prediction signature
signature = tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(
    inputs={"inputs": input_tensor},
    outputs={"score": output_tensor}
)


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


In [7]:
import tensorflow as tf

with tf.compat.v1.keras.backend.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    build.save()
    # build.add_meta_graph_and_variables(
    #     sess=sess, tags=[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY], 
    #     signature_def_map={"serving_default": signature})
    # build.save()


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: export/Servo/1\saved_model.pb


INFO:tensorflow:SavedModel written to: export/Servo/1\saved_model.pb


In [12]:
ls 'C:\\Users\mshee\OneDrive\Desktop\proo\export\Servo'

The filename, directory name, or volume label syntax is incorrect.


In [13]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [14]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\mshee\AppData\Local\sagemaker\sagemaker\config.yaml


In [15]:
# Create a new file named train.py
with open('train.py', 'w') as f:
    pass


In [18]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

In [19]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m5.large')

----!CPU times: total: 3.75 s
Wall time: 4min 24s


In [20]:
predictor.endpoint

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-tensorflow-serving-2024-04-06-18-33-45-013'

In [8]:
endpoint_name = 'sagemaker-tensorflow-serving-2024-03-22-08-38-43-072'